Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
import numpy as np
import pandas as pd
from scipy import stats

n = 160
mean_expected = 9.5
std = 0.4
mean = 9.57
z_stat = (mean - mean_expected) / (std / np.sqrt(n))
print(z_stat)
p = 2 * (1 - stats.norm.cdf(z_stat))
print(round(p, 4))

2.213594362117875
0.0269


Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

1. линейную регрессию с помощью LinearRegression без параметров
2. случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости \alpha=0.05α=0.05?

In [4]:
data = pd.read_table('diamonds.txt')
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [7]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.25, random_state=1)
X_train = data_train.drop(columns=['price'])
X_test = data_test.drop(columns=['price'])
X_train.head()

,carat,depth,table,x,y,z
41944,0.35,62.4,58.0,4.47,4.54,2.81
21479,1.52,62.9,59.0,7.31,7.26,4.58
4451,0.90,61.9,56.0,6.18,6.23,3.84
24948,1.70,62.1,57.0,7.61,7.55,4.71
7281,1.00,61.0,62.0,6.41,6.34,3.89


In [8]:
y_train = data_train['price']
y_test = data_test['price']
y_train.head()

41944      504
21479     9576
4451      3619
24948    13417
7281      4200
Name: price, dtype: int64

In [13]:
from sklearn.linear_model import LinearRegression

model_1 = LinearRegression()
model_1.fit(X_train, y_train)
predict_1 = model_1.predict(X_test)
abs_change_1 = abs(predict_1 - y_test)
abs_change_1.sum()

12006725.759779148

In [14]:
from sklearn.ensemble import RandomForestRegressor

model_2 = RandomForestRegressor(n_estimators=10, random_state=1)
model_2.fit(X_train, y_train)
predict_2 = model_2.predict(X_test)
abs_change_2 = abs(predict_2 - y_test)
abs_change_2.sum()

10827383.17541847

In [19]:
print(stats.ttest_rel(abs_change_1, abs_change_2))

Ttest_relResult(statistic=13.017729783878696, pvalue=1.655174575138418e-38)


In [20]:
from statsmodels.stats.weightstats import DescrStatsW

print(DescrStatsW(abs_change_1 - abs_change_2).tconfint_mean())

(74.2872453259553, 100.62452098634381)
